In [ ]:
### Import Google Drive:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
### Installs:
!pip install wget
!pip install humanfriendly jsonpickle # For MegaDetector

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
### Imports:
import cv2
import os
import pandas as pd 
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import wget
import tensorflow as tf
import json
import torch
import torchvision
import math
from sklearn.model_selection import train_test_split
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision import models

In [ ]:
### CONSTANTS:
BASE_PATH = "/content/gdrive/MyDrive/Data_Mining/"
CLASSES = {0: 'empty', 1: 'deer', 2: 'moose', 3: 'squirrel', 4: 'rodent', 5: 'small_mammal', \
                6: 'elk', 7: 'pronghorn_antelope', 8: 'rabbit', 9: 'bighorn_sheep', 10: 'fox', 11: 'coyote', \
                12: 'black_bear', 13: 'raccoon', 14: 'skunk', 15: 'wolf', 16: 'bobcat', 17: 'cat',\
                18: 'dog', 19: 'opossum', 20: 'bison', 21: 'mountain_goat', 22: 'mountain_lion'}
CLAHE = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(8,8))
ANNONATIONS_URL = "https://lilablobssc.blob.core.windows.net/iwildcam2019/iWildCam_2019_Annotations.tar.gz"
DETECTION_URL = "https://github.com/microsoft/CameraTraps/releases/download/v5.0/md_v5b.0.0.pt"

In [ ]:
### Load train.csv:
trainDf = pd.read_csv(BASE_PATH + "train.csv")

# Example:
ex = trainDf[trainDf["file_name"] == "5998cfa4-23d2-11e8-a6a3-ec086b02610b.jpg"]
int(ex["category_id"].values)

19

In [ ]:
"""
### Extract training images:
zipPath = "/content/gdrive/MyDrive/iwildcam-2019-fgvc6.zip"
!unzip -j $zipPath "train_images.zip"

savePath = BASE_PATH + '_TRAIN_UNZIPPED'

if not os.path.exists(savePath):
  os.mkdir(savePath)

#savePath = '_DATA'

#if not os.path.exists(savePath):
#  os.mkdir(savePath)

!unzip "train_images.zip"
""" 

'\n### Extract training images:\nzipPath = "/content/gdrive/MyDrive/iwildcam-2019-fgvc6.zip"\n!unzip -j $zipPath "train_images.zip"\n\nsavePath = BASE_PATH + \'_TRAIN_UNZIPPED\'\n\nif not os.path.exists(savePath):\n  os.mkdir(savePath)\n\n#savePath = \'_DATA\'\n\n#if not os.path.exists(savePath):\n#  os.mkdir(savePath)\n\n!unzip "train_images.zip"\n'

In [ ]:
"""
for file in os.listdir('.'): # Files in current working dir
   print(file)
   if file.endswith('.jpg'):
    os.rename(file, '_DATA/' + file) 
"""

"\nfor file in os.listdir('.'): # Files in current working dir\n   print(file)\n   if file.endswith('.jpg'):\n    os.rename(file, '_DATA/' + file) \n"

In [ ]:
"""
### Pre-processing the training images:
savePath = BASE_PATH + '_TRAIN_PROCESSED'
if not os.path.exists(savePath):
  os.mkdir(savePath)

path = '_DATA'
imgNames = os.listdir(path)

for imgName in imgNames:
  imgPath = path + '/' + imgName
  img = cv2.imread(imgPath)

  img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)  # Convert to LAB to seperate luminance and chrominance
  img[:,:,0] = CLAHE.apply(img[:,:,0]) # Apply CLAHE the luminance (L) channel
  img = cv2.cvtColor(img, cv2.COLOR_LAB2RGB) # Convert image back to RGB

  #TODO: Denoise, WB

  finalSavePath = savePath + '/' + imgName
  cv2.imwrite(finalSavePath, img)
  """

In [ ]:
path = BASE_PATH + '_TRAIN_PROCESSED/'
imgNames = os.listdir(path)

for imgName in imgNames:
  os.rename(path + imgName, path + imgName[10:])

KeyboardInterrupt: ignored

In [ ]:
imgNames = os.listdir(BASE_PATH + '_TRAIN_PROCESSED')

for imgName in imgNames:
  imgData = trainDf[trainDf["file_name"] == imgName]
  imgCategory = int(imgData["category_id"].values)
  
  if imgCategory == 0:
    rndNum = np.random.randint(0, 4)

    if rndNum != 3: # 1/4 Probability to KEEP an image of class 'empty'
      os.remove(BASE_PATH + '_TRAIN_PROCESSED/' + imgName)



In [ ]:
### Prequisits for running Megadetector:
#wget.download(DETECTION_URL) # Download Megadetector v5.0b

# Git clones and checkout:
!git clone https://github.com/microsoft/CameraTraps
!git clone https://github.com/microsoft/ai4eutils
!git clone https://github.com/ultralytics/yolov5/
!cd yolov5 && git checkout c23a441c9df7ca9b1f275e8c8719c949269160d1

# Add paths to PYTHONPATH:
os.environ['PYTHONPATH'] += ":/content/ai4eutils"
os.environ['PYTHONPATH'] += ":/content/CameraTraps"
os.environ['PYTHONPATH'] += ":/content/yolov5"

# Arguments for run_detector_batch:
imageDir = BASE_PATH + '_TRAIN_PROCESSED'
jsonOutputPath = BASE_PATH + 'megaDetectorJson2.json'

### Running MegaDetector:
!python CameraTraps/detection/run_detector_batch.py md_v5b.0.0.pt $imageDir $jsonOutputPath --recursive --output_relative_filenames --quiet

Cloning into 'CameraTraps'...
remote: Enumerating objects: 14633, done.
remote: Counting objects: 100% (1555/1555), done.
remote: Compressing objects: 100% (606/606), done.
remote: Total 14633 (delta 1012), reused 1480 (delta 948), pack-reused 13078
Receiving objects: 100% (14633/14633), 169.70 MiB | 18.58 MiB/s, done.
Resolving deltas: 100% (8833/8833), done.
Cloning into 'ai4eutils'...
remote: Enumerating objects: 778, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 778 (delta 48), reused 45 (delta 34), pack-reused 694
Receiving objects: 100% (778/778), 2.59 MiB | 20.06 MiB/s, done.
Resolving deltas: 100% (463/463), done.
Cloning into 'yolov5'...
remote: Enumerating objects: 15287, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 15287 (delta 21), reused 29 (delta 6), pack-reused 15236
Receiving objects: 100% (15287/15287), 14.18 MiB | 4.36 MiB/s, done.